# Animation example

In [ ]:
#@title Mount Google Drive
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    outputs_path = "/content/gdrive/MyDrive/AI/StabilityAnimations"
    !mkdir -p $outputs_path
except:
    outputs_path = "."
print(f"Animations will be saved to {outputs_path}")

In [ ]:
#@title Connect to the Stability API

# Grab GRPC protobuf definitions from the animation branch
!git clone -b animation https://github.com/Stability-AI/api-interfaces.git

import grpc
import sys
sys.path.append("api-interfaces/gooseai/generation")
import generation_pb2 as generation
import generation_pb2_grpc as generation_grpc

# GRPC endpoint and engines
GRPC_HOST = "" #@param {type:"string"}
API_KEY = "" #@param {type:"string"}
GENERATE_ENGINE_ID = 'stable-diffusion-v1-5'
INPAINT_ENGINE_ID = 'stable-diffusion-v1-5'
TRANSFORM_ENGINE_ID = 'transform-server-v1'

def open_channel(host: str, api_key: str = None) -> generation_grpc.GenerationServiceStub:
    print(f"Opening channel {host}")
    if host.endswith(":443"):
        call_credentials = []
        call_credentials.append(grpc.access_token_call_credentials(api_key))
        channel_credentials = grpc.composite_channel_credentials(
            grpc.ssl_channel_credentials(), *call_credentials
        )
        channel = grpc.secure_channel(host, channel_credentials)
    else:
        channel = grpc.insecure_channel(host)
    return generation_grpc.GenerationServiceStub(channel)

# Connect to Stability API
stub = open_channel(GRPC_HOST, api_key=API_KEY)


In [ ]:
#@title Code definitions

import bisect
import cv2
import datetime
import json
import logging
import numpy as np
import os
import pandas as pd
import pathlib
import random
import re
import subprocess
import sys

from base64 import b64encode
from IPython import display
from PIL import Image
from tqdm import tqdm
from types import SimpleNamespace
from typing import List, Tuple


def guidance_from_string(str: str) -> generation.GuidancePreset:
    mappings = {
        "None": generation.GUIDANCE_PRESET_NONE,
        "Simple": generation.GUIDANCE_PRESET_SIMPLE,
        "FastBlue": generation.GUIDANCE_PRESET_FAST_BLUE,
        "FastGreen": generation.GUIDANCE_PRESET_FAST_GREEN,
    }
    repr = mappings.get(str, None)
    if repr is None:
        raise Exception("invalid guider provided")
    return repr

def image_gen(
    stub:generation_grpc.GenerationServiceStub, 
    width:int, height:int, 
    prompts:List[str], weights:List[str], 
    steps:int, seed:int, cfg_scale:float, 
    sampler: generation.DiffusionSampler,
    init_image:np.ndarray, init_strength:float,
    init_noise_scale: float = 1.0,
    guidance_preset: generation.GuidancePreset = generation.GUIDANCE_PRESET_NONE,
    guidance_cuts: int = 0,
    guidance_strength: float = 0.0,
) -> np.ndarray:

    p = [generation.Prompt(text=prompt, parameters=generation.PromptParameters(weight=weight)) for prompt,weight in zip(prompts, weights)]
    if init_image is not None:
        p.append(image_to_prompt(init_image))

    step_parameters = {
        "scaled_step": 0,
        "sampler": generation.SamplerParameters(cfg_scale=cfg_scale, init_noise_scale=init_noise_scale),
    }

    begin_schedule = 1.0 if init_image is None else 1.0-init_strength
    if begin_schedule != 1.0:
        step_parameters["schedule"] = generation.ScheduleParameters(
            start=begin_schedule
        )

    if guidance_preset is not generation.GUIDANCE_PRESET_NONE:
        guidance_prompt = None
        guiders = None
        if guidance_cuts:
            cutouts = generation.CutoutParameters(count=guidance_cuts)
        else:
            cutouts = None
        step_parameters["guidance"] = generation.GuidanceParameters(
            guidance_preset=guidance_preset,
            instances=[
                generation.GuidanceInstanceParameters(
                    guidance_strength=guidance_strength,
                    models=guiders,
                    cutouts=cutouts,
                    prompt=guidance_prompt,
                )
            ],
        )

    imageParams = {
        "height": height,
        "width": width,
        "seed": [seed],
        "steps": steps,
        "parameters": [generation.StepParameter(**step_parameters)],
    }
    rq = generation.Request(
        engine_id=GENERATE_ENGINE_ID,
        prompt=p,
        image=generation.ImageParameters(**imageParams)
    )        
    rq.image.transform.diffusion = sampler

    for resp in stub.Generate(rq, wait_for_ready=True):
        for artifact in resp.artifacts:
            if artifact.type == generation.ARTIFACT_IMAGE:
                nparr = np.frombuffer(artifact.binary, np.uint8)
                return cv2.imdecode(nparr, cv2.IMREAD_COLOR)

def image_inpaint(
    stub:generation_grpc.GenerationServiceStub, image:np.ndarray, mask:np.ndarray,
    prompts:List[str], weights:List[float], steps:int, seed:int, cfg_scale:float,
    blur_ks:int = 11
) -> np.ndarray:
    width, height = image.shape[1], image.shape[0]
    mask = cv2.GaussianBlur(mask, (blur_ks,blur_ks), 0)

    p = [generation.Prompt(text=prompt, parameters=generation.PromptParameters(weight=weight)) for prompt,weight in zip(prompts, weights)]
    p.extend([
        image_to_prompt(image),
        image_to_prompt_mask(mask)
    ])
    rq = generation.Request(
        engine_id=INPAINT_ENGINE_ID,
        prompt=p,
        image=generation.ImageParameters(height=height, width=width, steps=steps, seed=[seed])
    )
    rq.image.parameters.append(
        generation.StepParameter(
            schedule=generation.ScheduleParameters(start=0.99),
            sampler=generation.SamplerParameters(cfg_scale=cfg_scale)
        )
    )
    for resp in stub.Generate(rq, wait_for_ready=True):
        for artifact in resp.artifacts:
            if artifact.type == generation.ARTIFACT_IMAGE:
                nparr = np.frombuffer(artifact.binary, np.uint8)
                return cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    
    raise Exception(f"no image artifact returned from inpaint request")

def image_mix(img_a: np.ndarray, img_b: np.ndarray, tween: float) -> np.ndarray:
    assert(img_a.shape == img_b.shape)
    return (img_a.astype(float)*(1.0-tween) + img_b.astype(float)*tween).astype(img_a.dtype)

def image_to_jpg_bytes(image: np.ndarray, quality: int=90):
    return cv2.imencode('.jpg', image, [int(cv2.IMWRITE_JPEG_QUALITY), quality])[1].tobytes()

def image_to_png_bytes(image: np.ndarray):
    return cv2.imencode('.png', image)[1].tobytes()

def image_to_prompt(image: np.ndarray) -> generation.Prompt:
    return generation.Prompt(
        parameters=generation.PromptParameters(init=True),
        artifact=generation.Artifact(
            type=generation.ARTIFACT_IMAGE,
            binary=image_to_png_bytes(image)))

def image_to_prompt_mask(image: np.ndarray) -> generation.Prompt:
    mask = image_to_prompt(image)
    mask.artifact.type = generation.ARTIFACT_MASK
    return mask

def image_xform(
    stub:generation_grpc.GenerationServiceStub, 
    images:List[np.ndarray], 
    ops:List[generation.TransformOperation]
) -> Tuple[List[np.ndarray], np.ndarray]:
    assert(len(images))
    transforms = generation.TransformSequence(operations=ops)
    p = [image_to_prompt(image) for image in images]
    rq = generation.Request(
        engine_id=TRANSFORM_ENGINE_ID,
        prompt=p,
        image=generation.ImageParameters(transform=generation.TransformType(sequence=transforms)),
    )

    images, mask = [], None
    for resp in stub.Generate(rq, wait_for_ready=True):
        for artifact in resp.artifacts:
            if artifact.type == generation.ARTIFACT_IMAGE:
                nparr = np.frombuffer(artifact.binary, np.uint8)
                images.append(cv2.imdecode(nparr, cv2.IMREAD_COLOR))
            elif artifact.type == generation.ARTIFACT_MASK:
                nparr = np.frombuffer(artifact.binary, np.uint8)
                mask = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    return images, mask

def key_frame_inbetweens(key_frames, max_frames, integer=False, interp_method='Linear'):
    key_frame_series = pd.Series([np.nan for a in range(max_frames)])

    for i, value in key_frames.items():
        key_frame_series[i] = value
    key_frame_series = key_frame_series.astype(float)
    
    if interp_method == 'Cubic' and len(key_frames.items()) <= 3:
      interp_method = 'Quadratic'    
    if interp_method == 'Quadratic' and len(key_frames.items()) <= 2:
      interp_method = 'Linear'
          
    key_frame_series[0] = key_frame_series[key_frame_series.first_valid_index()]
    key_frame_series[max_frames-1] = key_frame_series[key_frame_series.last_valid_index()]
    key_frame_series = key_frame_series.interpolate(method=interp_method.lower(), limit_direction='both')
    if integer:
        return key_frame_series.astype(int)
    return key_frame_series

def key_frame_parse(string, prompt_parser=None):
    pattern = r'((?P<frame>[0-9]+):[\s]*[\(](?P<param>[\S\s]*?)[\)])'
    frames = dict()
    for match_object in re.finditer(pattern, string):
        frame = int(match_object.groupdict()['frame'])
        param = match_object.groupdict()['param']
        if prompt_parser:
            frames[frame] = prompt_parser(param)
        else:
            frames[frame] = param
    if frames == {} and len(string) != 0:
        raise RuntimeError('Key Frame string not correctly formatted')
    return frames

def sampler_from_string(str: str) -> generation.DiffusionSampler:
    mappings = {
        "DDIM": generation.SAMPLER_DDIM,
        "PLMS": generation.SAMPLER_DDPM,
        "K_euler": generation.SAMPLER_K_EULER,
        "K_euler_ancestral": generation.SAMPLER_K_EULER_ANCESTRAL,
        "K_heun": generation.SAMPLER_K_HEUN,
        "K_dpm_2": generation.SAMPLER_K_DPM_2,
        "K_dpm_2_ancestral": generation.SAMPLER_K_DPM_2_ANCESTRAL,
        "K_lms": generation.SAMPLER_K_LMS,
    }
    repr = mappings.get(str, None)
    if not repr:
        raise Exception("invalid sampler provided")
    return repr

def warp2d_op(dx:float, dy:float, rotate:float, scale:float, border:str) -> generation.TransformOperation:
    warp2d = generation.TransformWarp2d()

    if border == 'replicate': warp2d.border_mode = generation.BORDER_REPLICATE
    elif border == 'reflect': warp2d.border_mode = generation.BORDER_REFLECT
    elif border == 'wrap': warp2d.border_mode = generation.BORDER_WRAP
    elif border == 'zero': warp2d.border_mode = generation.BORDER_ZERO
    else: raise Exception(f"invalid 2d border mode {border}")

    warp2d.rotate = rotate
    warp2d.scale = scale
    warp2d.translate_x = dx
    warp2d.translate_y = dy
    return generation.TransformOperation(warp2d=warp2d)

def warp3d_op(
    dx:float, dy:float, dz:float, rx:float, ry:float, rz:float,
    near:float, far:float, fov:float, border:str
) -> generation.TransformOperation:
    warp3d = generation.TransformWarp3d()

    if border == 'replicate': warp3d.border_mode = generation.BORDER_REPLICATE
    elif border == 'reflect': warp3d.border_mode = generation.BORDER_REFLECT
    elif border == 'zero': warp3d.border_mode = generation.BORDER_ZERO
    else: raise Exception(f"invalid 3d border mode {border}")

    warp3d.translate_x = dx
    warp3d.translate_y = dy
    warp3d.translate_z = dz
    warp3d.rotate_x = rx
    warp3d.rotate_y = ry
    warp3d.rotate_z = rz
    warp3d.near_plane = near
    warp3d.far_plane = far
    warp3d.fov = fov
    return generation.TransformOperation(warp3d=warp3d)



In [ ]:
#@title Settings

def Args():

    #@markdown ####**Settings:**
    W = 512 #@param
    H = 512 #@param
    W, H = map(lambda x: x - x % 64, (W, H))  # resize to integer multiple of 64
    sampler = 'K_dpm_2_ancestral' #@param ["DDIM", "PLMS", "K_euler", "K_euler_ancestral", "K_heun", "K_dpm_2", "K_dpm_2_ancestral", "K_lms"]
    seed = 42 #@param
    cfg_scale = 7 #@param {type:"number"}
    clip_guidance = 'FastBlue' #@param ["None", "Simple", "FastBlue", "FastGreen"]

    #@markdown ####**Animation Settings:**
    animation_mode = '3D' #@param ['2D', '3D', 'Video Input'] {type:'string'}
    max_frames = 60 #@param {type:"number"}
    border = 'replicate' #@param ['reflect', 'replicate', 'wrap', 'zero'] {type:'string'}
    inpaint_border = False #@param {type:"boolean"}
    interpolate_prompts = False #@param {type:"boolean"}
    locked_seed = False #@param {type:"boolean"}

    #@markdown ####**Key framed value curves:**
    angle = "0:(1)" #@param {type:"string"}
    zoom = "0:(1.05)" #@param {type:"string"}
    translation_x = "0:(0)" #@param {type:"string"}
    translation_y = "0:(0)" #@param {type:"string"}
    translation_z = "0:(5)" #@param {type:"string"}
    rotation_x = "0:(0)" #@param {type:"string"}
    rotation_y = "0:(0)" #@param {type:"string"}
    rotation_z = "0:(1)" #@param {type:"string"}
    brightness_curve = "0: (1.0)" #@param {type:"string"}
    contrast_curve = "0: (1.0)" #@param {type:"string"}
    noise_curve = "0:(0.0)" # likely to be removed, still hidden here for potential experiments
    noise_scale_curve = "0:(1.02)" #@param {type:"string"}
    steps_curve = "0:(50)" #@param {type:"string"}
    strength_curve = "0:(0.65)" #@param {type:"string"}

    #@markdown ####**Coherence:**
    color_coherence = True #@param {type:"boolean"}
    diffusion_cadence = '3' #@param ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16'] {type:'string'}

    #@markdown ####**3D Depth Warping:**
    #use_depth_warping = True #@param {type:"boolean"}
    midas_weight = 0.3 #@param {type:"number"}
    near_plane = 200
    far_plane = 10000
    fov = 20 #@param {type:"number"}
    save_depth_maps = False #@param {type:"boolean"}

    #@markdown ####**Video Input:**
    video_init_path ='/content/video_in.mp4' #@param {type:"string"}
    extract_nth_frame = 4 #@param {type:"number"}
    video_mix_in = 0.02 #@param {type:"number"}
    video_flow_warp = True #@param {type:"boolean"}

    return locals()


### Prompts

In [ ]:
animation_prompts = {
    0: "a painting of a delicious cheeseburger by Tyler Edlin",
    24: "a painting of the the answer to life the universe and everything by Tyler Edlin",
}

negative_prompt = ""
negative_prompt_weight = -1.0

In [ ]:
#@title Render the animation

def display_frame(image: np.ndarray):
    display.clear_output(wait=True)
    display.display(Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)))

def get_animation_prompts_weights(frame_idx: int, key_frame_values: List[int], interp: bool) -> Tuple[List[str], List[float]]:
    idx = bisect.bisect_right(key_frame_values, frame_idx)
    prev, next = idx - 1, idx
    if not interp:
        return [animation_prompts[key_frame_values[min(len(key_frame_values)-1, prev)]]], [1.0]
    elif next == len(key_frame_values):
        return [animation_prompts[key_frame_values[-1]]], [1.0]
    else:
        tween = (frame_idx - key_frame_values[prev]) / (key_frame_values[next] - key_frame_values[prev])
        return [animation_prompts[key_frame_values[prev]], animation_prompts[key_frame_values[next]]], [1.0 - tween, tween]

def render_animation(args, out_dir):
    # choose a random seed if set to 0
    if args.seed == 0:
        args.seed = random.randint(0, 2**32 - 1)

    # save settings for the animation
    settings_filename = os.path.join(out_dir, f"{timestring}_settings.txt")
    with open(settings_filename, "w+", encoding="utf-8") as f:
        json.dump(vars(args), f, ensure_ascii=False, indent=4)

    # expand key frames
    angle_series = key_frame_inbetweens(key_frame_parse(args.angle), args.max_frames)
    zoom_series = key_frame_inbetweens(key_frame_parse(args.zoom), args.max_frames)
    translation_x_series = key_frame_inbetweens(key_frame_parse(args.translation_x), args.max_frames)
    translation_y_series = key_frame_inbetweens(key_frame_parse(args.translation_y), args.max_frames)
    translation_z_series = key_frame_inbetweens(key_frame_parse(args.translation_z), args.max_frames)
    rotation_x_series = key_frame_inbetweens(key_frame_parse(args.rotation_x), args.max_frames)
    rotation_y_series = key_frame_inbetweens(key_frame_parse(args.rotation_y), args.max_frames)
    rotation_z_series = key_frame_inbetweens(key_frame_parse(args.rotation_z), args.max_frames)
    brightness_series = key_frame_inbetweens(key_frame_parse(args.brightness_curve), args.max_frames)
    contrast_series = key_frame_inbetweens(key_frame_parse(args.contrast_curve), args.max_frames)
    noise_series = key_frame_inbetweens(key_frame_parse(args.noise_curve), args.max_frames)
    noise_scale_series = key_frame_inbetweens(key_frame_parse(args.noise_scale_curve), args.max_frames)
    steps_series = key_frame_inbetweens(key_frame_parse(args.steps_curve), args.max_frames)
    strength_series = key_frame_inbetweens(key_frame_parse(args.strength_curve), args.max_frames)

    # prepare sorted list of key frames
    key_frame_values = sorted(list(animation_prompts.keys()))
    if key_frame_values[0] != 0:
        raise ValueError("First keyframe must be 0")
    if len(key_frame_values) != len(set(key_frame_values)):
        raise ValueError("Duplicate keyframes are not allowed!")

    # diffusion performed every N frames. two prior diffused frames
    # are transformed and blended between to produce each output frame
    diffusion_cadence = max(1, int(args.diffusion_cadence))
    prior_frames = []

    # load input video
    video_in = args.video_init_path if args.animation_mode == 'Video Input' else None
    video_reader = None if video_in is None else cv2.VideoCapture(video_in)
    video_extract_nth = args.extract_nth_frame
    video_prev_frame = None
    if video_reader is not None:
        success, image = video_reader.read()
        if not success:
            raise Exception(f"Failed to read first frame from {video_in}")
        video_prev_frame = cv2.resize(image, (args.W, args.H), interpolation=cv2.INTER_LANCZOS4)
        prior_frames = [video_prev_frame, video_prev_frame]

    color_match_image = None # optional target for color matching
    inpaint_mask = None      # optional mask of revealed areas
    seed = args.seed

    for frame_idx in tqdm(range(args.max_frames)):
        steps = int(steps_series[frame_idx])

        # fetch set of prompts and weights for this frame
        prompts, weights = get_animation_prompts_weights(frame_idx, key_frame_values, interp=args.interpolate_prompts)
        if len(negative_prompt) and negative_prompt_weight != 0.0:
            prompts.append(negative_prompt)
            weights.append(-abs(negative_prompt_weight))

        # apply transformation to prior frames
        if len(prior_frames):
            ops = []
            if args.save_depth_maps or args.animation_mode == '3D':
                ops.append(generation.TransformOperation(                    
                    depth_calc=generation.TransformDepthCalc(
                        blend_weight=args.midas_weight,
                        export=args.save_depth_maps
                    )
                ))
            if args.animation_mode == '2D':
                ops.append(warp2d_op(
                    translation_x_series[frame_idx], 
                    translation_y_series[frame_idx], 
                    angle_series[frame_idx], 
                    zoom_series[frame_idx], 
                    args.border
                ))
            elif args.animation_mode == '3D':
                ops.append(warp3d_op(
                    translation_x_series[frame_idx], 
                    translation_y_series[frame_idx], 
                    translation_z_series[frame_idx], 
                    rotation_x_series[frame_idx], 
                    rotation_y_series[frame_idx], 
                    rotation_z_series[frame_idx], 
                    args.near_plane, args.far_plane, 
                    args.fov, args.border
                ))
            elif args.animation_mode == 'Video Input':
                for i in range(video_extract_nth):
                    success, video_next_frame = video_reader.read()
                if success:
                    video_next_frame = cv2.resize(video_next_frame, (args.W, args.H), interpolation=cv2.INTER_LANCZOS4)
                    if args.video_flow_warp:
                        ops.append(generation.TransformOperation(
                            warp_flow=generation.TransformWarpFlow(
                                prev_frame=generation.Artifact(type=generation.ARTIFACT_IMAGE, binary=image_to_jpg_bytes(video_prev_frame)),
                                next_frame=generation.Artifact(type=generation.ARTIFACT_IMAGE, binary=image_to_jpg_bytes(video_next_frame)),
                            )
                        ))
                    video_prev_frame = video_next_frame
                    color_match_image = video_next_frame
            if len(ops):
                prior_frames, mask = image_xform(stub, prior_frames, ops)
                inpaint_mask = mask if args.inpaint_border else None

                depth_map = prior_frames.pop(0) if len(prior_frames) == 3 else None
                if depth_map is not None and args.save_depth_maps:
                    cv2.imwrite(os.path.join(out_dir, f"depth_{frame_idx:05d}.png"), depth_map)

                if inpaint_mask is not None:
                    for i in range(len(prior_frames)):
                        prior_frames[i] = image_inpaint(stub, prior_frames[i], inpaint_mask, prompts, weights, steps//2, seed, args.cfg_scale)
                    inpaint_mask = None

        # either run diffusion or emit an inbetween frame
        if frame_idx % diffusion_cadence == 0:
            if inpaint_mask is not None:
                prior_frames[-1] = image_inpaint(stub, prior_frames[-1], inpaint_mask, prompts, weights, steps//2, seed, args.cfg_scale)
                inpaint_mask = None
            strength = strength_series[frame_idx]

            # apply additional noising and color matching to previous frame to use as init
            init_image = prior_frames[-1] if len(prior_frames) and strength > 0 else None
            if init_image is not None:
                noise = noise_series[frame_idx]
                brightness = brightness_series[frame_idx]
                contrast = contrast_series[frame_idx]
                ops = []
                if args.color_coherence and color_match_image is not None:                    
                    ops.append(generation.TransformOperation(color_match=generation.TransformColorMatch(
                        color_mode=generation.COLOR_MATCH_LAB,
                        image=generation.Artifact(type=generation.ARTIFACT_IMAGE, binary=image_to_jpg_bytes(color_match_image))
                    )))
                if args.video_mix_in > 0 and video_prev_frame is not None:
                    ops.append(generation.TransformOperation(blend=generation.TransformBlend(
                        amount=args.video_mix_in, 
                        target=generation.Artifact(type=generation.ARTIFACT_IMAGE, binary=image_to_jpg_bytes(video_prev_frame))
                    )))
                if brightness != 1.0 or contrast != 1.0:
                    ops.append(generation.TransformOperation(contrast=generation.TransformContrast(
                        brightness=brightness, contrast=contrast
                    )))
                if noise > 0:
                    ops.append(generation.TransformOperation(add_noise=generation.TransformAddNoise(amount=noise, seed=seed)))
                if len(ops):
                    init_image = image_xform(stub, [init_image], ops)[0][0]

            # generate the next frame
            sampler = sampler_from_string(args.sampler)
            guidance = guidance_from_string(args.clip_guidance)
            noise_scale = noise_scale_series[frame_idx]
            image = image_gen(
                stub, 
                args.W, args.H, 
                prompts, weights, 
                steps, seed, args.cfg_scale, sampler, 
                init_image, strength,
                init_noise_scale=noise_scale, 
                guidance_preset=guidance
            )

            if color_match_image is None:
                color_match_image = image
            if not len(prior_frames):
                prior_frames = [image, image]
            
            cv2.imwrite(os.path.join(out_dir, f'frame_{frame_idx:05}.png'), prior_frames[1])
            display_frame(prior_frames[1])
            prior_frames[0] = prior_frames[1]
            prior_frames[1] = image            
        else:
            # smoothly blend between prior frames
            tween = (frame_idx % diffusion_cadence) / float(diffusion_cadence)
            t = image_mix(prior_frames[0], prior_frames[1], tween)
            cv2.imwrite(os.path.join(out_dir, f'frame_{frame_idx:05}.png'), t)
            display_frame(t)

        if not args.locked_seed:
            seed += 1

# create folder for frames output
timestring = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
out_dir = os.path.join(outputs_path, timestring)
os.makedirs(out_dir, exist_ok=True)
print(f"Saving animation frames to {out_dir}...")

render_animation(SimpleNamespace(**Args()), out_dir)

In [ ]:
#@title Create video from frames

fps = 12 #@param {type:"number"}

image_path = os.path.join(out_dir, "frame_%05d.png")
mp4_path = os.path.join(out_dir, f"{timestring}.mp4")

print(f"Compiling animation frames to {mp4_path}...")

cmd = [
    'ffmpeg',
    '-y',
    '-vcodec', 'png',
    '-r', str(fps),
    '-start_number', str(0),
    '-i', image_path,
    '-c:v', 'libx264',
    '-vf',
    f'fps={fps}',
    '-pix_fmt', 'yuv420p',
    '-crf', '17',
    '-preset', 'veryfast',
    mp4_path
]
process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    print(stderr)
    raise RuntimeError(stderr)

mp4 = open(mp4_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
display.display( display.HTML(f'<video controls loop><source src="{data_url}" type="video/mp4"></video>') )
